# 1. IMPORT

In [6]:
import json 
import pandas as pd
from collections import deque

termid = 55 # legislatura
start_date = '2015-01-02'

PROV = 'cam'
JSON_PATH =  'datasets/camara/json/'

AGENTS_PATH = 'datasets/slp/agents/'
MEMBERSHIPS_PATH = 'datasets/slp/memberships/'
ORGANIZATIONS_PATH = 'datasets/slp/organizations/'
POSTS_PATH = 'datasets/slp/posts/'

FILE_SUFFIX = '55'
CAMARA_URI = 'c8f660fd-7d0b-48e5-8513-2f7f0bb5f91c' 


 # 1.2 Getting the parties

In [7]:
filename = 'parties.csv'
file_path = '{:}{:}'.format(ORGANIZATIONS_PATH, filename)
df_parties = pd.read_csv(file_path, sep= ';', encoding= 'utf-8', index_col=None)
df_parties.set_index('sigla', inplace=True)


parties_d = df_parties['slp:resource_uri'].to_dict()                        
print({k:parties_d[k]
    for k in ['PT', 'PMDB', 'DEM']})

{'PT': '7b2138a6-df61-11e7-baf1-c82a144c0a85', 'PMDB': '7b210cfa-df61-11e7-9864-c82a144c0a85', 'DEM': '7b20ed7e-df61-11e7-bb65-c82a144c0a85'}


# 2. Reads json and stores in an array of dicts

In [16]:
filename = 'congressman_with_memberships-{:}.json'.format(FILE_SUFFIX) 
file_path = '{:}{:}'.format(JSON_PATH, filename)
with open(file_path, mode='r') as f:
    senatorsstr = f.read()
f.close()

congressmen_with_memberships = json.loads(senatorsstr)

# 3. Iterates saving the agents

In [17]:
mapping = {'skos:prefLabel': 'cam:ideCadastro',
  'foaf:name':'cam:nomeCivil',
  'rdfs:label':'cam:nomeParlamentarAtual',
  'agent_resource_uri':'slp:resource_uri'           
}

agents = [] 
for cwm_d in congressmen_with_memberships:
    agent_d = {mapping[k]: cwm_d.get(k, None) for k in mapping}
    if agent_d:
        agents.append(agent_d)

filename = 'congressmen-{:}-{:}.csv'.format(PROV, FILE_SUFFIX) 
file_path = '{:}{:}'.format(AGENTS_PATH, filename)
df = pd.DataFrame.from_dict(agents)    
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)
df.head()

,cam:ideCadastro,cam:nomeCivil,cam:nomeParlamentarAtual,slp:resource_uri
0,178943,MARCOS BEZERRA RIBEIRO SOARES,MARCOS SOARES,1a2a8c12348380555e510d1e1b6e7de3
1,178995,ANTONIO ROBERTO RODRIGUES GÓES DA SILVA,ROBERTO GÓES,d756776bc56efce11d2fecee1f4855d0
2,112437,JOÃO MARCELO SANTOS SOUZA,JOÃO MARCELO SOUZA,de549e980bd660b3b287d5893a2e41a8
3,160544,JUNJI ABE,JUNJI ABE,955f88ef-358e-46a3-8f8c-a67be902240c
4,160544,JUNJI ABE,JUNJI ABE,955f88ef-358e-46a3-8f8c-a67be902240c


 # 4.1 Iterates saving memberships to the congress

In [19]:
mapping = {
    'skos:prefLabel': 'cam:ideCadastro',
    'natureza': 'cam:siglaUFRepresentacao',
    'startDate': 'cam:dataInicio', 
    'finishDate': 'cam:dataFim',
    'membership_resource_uri':'slp:resource_uri',
    'role_resource_uri':'org:role'
}

memberships = [] 
for cwm_d in congressmen_with_memberships:
    terms = cwm_d['terms']
    if terms:
        for term_d in terms:
            memberships_d = {mapping[k]: term_d.get(k, None) for k in mapping}
            if memberships_d:
                memberships.append(memberships_d)


filename = 'memberships_with_congress-{:}-{:}.csv'.format(PROV, FILE_SUFFIX) 
file_path = '{:}{:}'.format(MEMBERSHIPS_PATH, filename)
df = pd.DataFrame.from_dict(memberships)    
# df.to_csv(file_path, sep=';', encoding='utf-8', index=None)
print(df.shape[0])
df.head()            

2700


,cam:dataFim,cam:dataInicio,cam:ideCadastro,cam:siglaUFRepresentacao,org:role,slp:resource_uri
0,None,2015-02-01,None,RJ,b27beba7-ca02-4041-a9e0-1793bcd141fe,e24c70a0-7f31-46af-94f1-faa0e8dc05fa
1,None,2015-02-01,None,AP,b27beba7-ca02-4041-a9e0-1793bcd141fe,fe1e6247-f350-4072-a3c0-25313737ba48
2,None,2015-02-01,None,MA,b27beba7-ca02-4041-a9e0-1793bcd141fe,2e998dda-f39b-463d-95a5-83725fde42f2
3,2015-01-31,2011-02-01,None,SP,b27beba7-ca02-4041-a9e0-1793bcd141fe,3d3daf3b-106c-420c-b2a7-971bb73dd974
4,2015-01-31,2011-02-01,None,SP,b27beba7-ca02-4041-a9e0-1793bcd141fe,3d3daf3b-106c-420c-b2a7-971bb73dd974
